# `focus.mrcz` Tutorial

## Getting started

For this tutorial, let's download [EMD-4628](https://www.ebi.ac.uk/pdbe/entry/emdb/EMD-4628) as an example for using [MRCZ](https://github.com/em-MRCZ/python-mrcz) compression/decompression.  
Although we will be working with a 3D map for simplicity here (and MRCZ comes in very handy when handling projects with _many_ 3D maps!), it can also be used on __2D micrographs__ or __movies__.

(**NOTE:** internally, EMDB stores 3D maps in the MAP format, which is almost identical to the MRC format except the internal coordinates are stored in a different order - nothing to worry about here)

In [1]:
%%bash

# Download the EMDB-4628 map:
wget ftp://ftp.ebi.ac.uk/pub/databases/emdb/structures/EMD-4628/map/emd_4628.map.gz

# Unzip it:
gunzip emd_4628.map.gz

--2019-09-24 15:41:22--  ftp://ftp.ebi.ac.uk/pub/databases/emdb/structures/EMD-4628/map/emd_4628.map.gz
           => ‘emd_4628.map.gz’
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.197.74
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.197.74|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/emdb/structures/EMD-4628/map ... done.
==> SIZE emd_4628.map.gz ... 34766556
==> PASV ... done.    ==> RETR emd_4628.map.gz ... done.
Length: 34766556 (33M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  0%  924K 37s
    50K .......... .......... .......... .......... ..........  0% 2.11M 26s
   100K .......... .......... .......... .......... ..........  0% 89.9M 18s
   150K .......... .......... .......... .......... ..........  0%  115M 13s
   200K .......... .......... .......... .......... ..........  0% 2.15M 14s
   250K .......... .......... .......... .

## Compressing

### Default options

Compressing a MAP/MRC file is very very easy with the default options (compressor: zstd):

**NOTE:** The warning message `No handlers could be found for logger "mrcz.test_mrcz"` can be safely ignored.

In [2]:
!focus.mrcz emd_4628.map

No handlers could be found for logger "mrcz.test_mrcz"


Now let's compare the size of our original file (`emd_4628.map`) with the compressed file (`emd_4628.mrcz`):

In [3]:
%%bash
\ls -sh emd_4628.map
\ls -sh emd_4628.mrcz

283M emd_4628.map
30M emd_4628.mrcz


A pretty decent compression ratio of ~0.10 :-)  
Note, however, the beauty of MRCZ is not only the compression _ratio_ but also the (de)compression _rate_ (i.e. speed). Usually, the latter only becomes apparent if you are compressing or decompressing large amounts of data (e.g. in a script).

### Exploring the compressors

Now, let's compress our map using all the compressing algorithms provided by the [Blosc](http://blosc.org/) library via MRCZ:

In [4]:
%%bash

focus.mrcz emd_4628.map --compressor blosclz --suffix blosclz
focus.mrcz emd_4628.map --compressor lz4 --suffix lz4
focus.mrcz emd_4628.map --compressor lz4hc --suffix lz4hc
focus.mrcz emd_4628.map --compressor zlib --suffix zlib
focus.mrcz emd_4628.map --compressor zstd --suffix zstd

# Now let's compare their size:
\ls -Ssh emd_4628_* | cat

108M emd_4628_blosclz.mrcz
 36M emd_4628_lz4.mrcz
 34M emd_4628_lz4hc.mrcz
 33M emd_4628_zlib.mrcz
 30M emd_4628_zstd.mrcz


No handlers could be found for logger "mrcz.test_mrcz"
No handlers could be found for logger "mrcz.test_mrcz"
No handlers could be found for logger "mrcz.test_mrcz"
No handlers could be found for logger "mrcz.test_mrcz"
No handlers could be found for logger "mrcz.test_mrcz"


## Decompressing

Decompressing is also very easy:

In [5]:
!focus.mrcz emd_4628.mrcz --decompress 

No handlers could be found for logger "mrcz.test_mrcz"


Note the output file will be saved with the `.mrc` extension even though it was originally a `.map` file.  
We can now compare the size of the uncompressed file with the original file and verify they are identical (except maybe for a few header bytes):

In [6]:
%%bash

\ls -s emd_4628.map emd_4628.mrc | cat

289412 emd_4628.map
289408 emd_4628.mrc


## Operating on multiple files

`focus.mrcz` also accepts multiple input files at once, for example:

`focus.mrcz *.mrc`

would generate a compressed `.mrcz` file for every `.mrc` input (using the default options in this case).